In [3]:
import urllib.request
from bs4 import BeautifulSoup as bs
import re, json
from IPython.display import clear_output
import threading
import sys
import queue
import datetime
import time

# CONFIG
JSON_FILE_LOCATION = 'tweet_data_original.json'
NUM_OF_THREADS = 20

# EVIL GLOBALS
status = (0, 0, None, None, None)
EOF = 'EOF'
f = open(JSON_FILE_LOCATION, encoding='utf8')
lock = threading.Lock()
input_lock = threading.Lock()
q = queue.Queue()
passed = 0
failed = 0

def chunkify(lst,n):
    return [lst[i::n] for i in range(n)]

def get_tweet():
    global f
    with input_lock:
        try:
            while(True):
                line = f.readline()
                if line == '':
                    return EOF
                tweet = json.loads(line)
                if len(tweet['entities']['urls']) > 0:
                    #print(tweet['entities']['urls'][0]['url'])
                    return tweet
        except:
            return EOF
    
def visible(element):
    if element.parent.name in ['style', 'script', '[document]', 'head', 'title']:
        return False
    elif re.match('<!--.*-->', str(element)) or re.match('<![endif].*', str(element)):
        return False
    return True

    
def main():
    threads = []
    qt = threading.Thread(target=queue_worker)
    qt.daemon = True
    qt.start()
    
    for i in range(NUM_OF_THREADS):
        t = threading.Thread(target=tweet_link_scraper_worker)
        t.daemon = True
        t.start()
        threads.append(t)
        
    while True:
        finished = True
        for thread in threads:
            if thread.isAlive():
                finished = False
                
        clear_output(wait=True)
        print("Passed: {}, Failed: {}, Time elapsed: {}, items/second:  {} \n{}".format(status[0], status[1], status[2], status[3], status[4]))
        sys.stdout.flush()
        time.sleep(2)
        if finished:
            break
    
    q.join()
    print("Done.")
    f.close()
    
def queue_worker():
    global status
    start_time = datetime.datetime.now()
    passed = 0
    failed = 0
    while True:
        item = q.get()
        error = do_queue_work(item)
        if error == 0:
            passed += 1
        else:
            failed += 1
        time_delta = datetime.datetime.now() - start_time
        seconds_elapsed = time_delta.total_seconds() if time_delta.total_seconds() > 0 else 1
        average_item_per_second = int((passed + failed)/seconds_elapsed)
        #clear_output()
        status = (passed, failed, time_delta, average_item_per_second, item['url'])
        q.task_done()
        
def do_queue_work(item):
    if item == None:
        return 1
    else:
        return 0

def tweet_link_scraper_worker():
    global passed, failed, total
    while(True):
        doc = {}
        tweet = get_tweet()
        data = None
        if tweet == EOF:
            break
        else:
            try:
                with urllib.request.urlopen(tweet['entities']['urls'][0]['url']) as response:
                    html = response.read()
                    url = response.geturl()
                    soup = bs(html, 'html.parser')
                    text = soup.findAll(text=True)
                    text_items = filter(visible, text)
                    visible_text = [item.strip() for item in text_items if item != '\n' and len(item.strip().split(' ')) > 10]
                    
                    doc['texts'] = visible_text
                    doc['images'] = [tag['src'] for tag in bs.findAll(itemprop='image')]
                    doc['url'] = url
                    
            except:
                # Could not access address
                # print(tweet['entities']['urls'][0]['url'])
                pass

            finally:
                q.put(doc)

if __name__ == '__main__':
    main()

Passed: 0, Failed: 0, Time elapsed: None, items/second:  None 
None


KeyboardInterrupt: 